In [17]:
import pandas as pd
import numpy as np
from keras.layers import Dense,Dropout,Activation,Input
from keras.models import Sequential,Model
from keras.callbacks import EarlyStopping, BaseLogger
from sklearn.model_selection import train_test_split
from keras import metrics
import tensorflow as tf
import keras.backend.tensorflow_backend as KTF
import os
#进行配置，每个GPU使用60%上限现存
os.environ["CUDA_VISIBLE_DEVICES"]="1" # 使用编号为1，2号的GPU
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.6 # 每个GPU现存上届控制在60%以内
session = tf.Session(config=config)

# 设置session
KTF.set_session(session )

In [41]:
df_train = pd.read_csv('data/train_1.csv')
y_train = pd.read_csv('data/label_1.csv').values
df_test = pd.read_csv('data/test_1.csv')

In [ ]:
model = Sequential()
input_size = len(df_train.columns)
model.add(Dense(units=90, activation='relu', input_shape=(input_size, )))
model.add(Dropout(0.5))
model.add(Dense(units=45, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(units=30,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=15, activation='relu'))
model.add(Dropout(0.1))
# 此处不能使用激活函数，因为放假是放射的
model.add(Dense(units=1,activation=None))
# 官网使用mse计算损失
model.compile(loss='mean_squared_error',optimizer='adam',metrics=[metrics.mae])
model.summary()

In [ ]:
x_tra, x_val, y_tra, y_val = train_test_split(df_train, y_train, test_size=0.2, random_state=2019)

In [ ]:
history = model.fit(x_tra, y_tra, batch_size=64, epochs=5000, verbose=1, validation_data=(x_val, y_val),)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(np.arange(len(history.history['loss'])), history.history['loss'], label='train')
plt.plot(np.arange(len(history.history['val_loss'])), history.history['val_loss'], label='valid')
plt.xlabel('epochs')
plt.ylabel('losses')
plt.legend(loc=0)
plt.show()

In [ ]:
result = pd.DataFrame({'price': model.predict(df_test).reshape(1, -1)[0]})
result.to_csv('result.csv', index=False, encoding='utf-8')